This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [4]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [22]:
# First, import the relevant modules
import requests


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [32]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)
print(r.status_code, r.headers['content-type'])

200 application/json; charset=utf-8


In [36]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()['dataset_data']['column_names'], r.json()['dataset_data']['data'][0]

(['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 ['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [107]:
import heapq

data = r.json()['dataset_data']
columns = data['column_names']
#print(columns)

highest_open_price = 0
lowest_open_price = 100000
largest_change = 0
total_volume = 0
total_days = 0
trading_volume = []
prev_close = 0
largest_change2 = 0
minheap = []
maxheap = []
middle = len(data['data'])//2

for entry in data['data'][::-1]:
    if entry[1] and entry[1]>highest_open_price: highest_open_price = entry[1]
    if entry[1] and entry[1]<lowest_open_price: lowest_open_price = entry[1]
    temp = entry[2]-entry[3]
    if temp > largest_change:  largest_change = temp
    temp = entry[4]-prev_close
    if temp > largest_change2:   largest_change2 = temp
    prev_close = entry[4]
    total_volume += entry[6]
    total_days += 1
    
    if len(maxheap)<middle:     heapq.heappush(maxheap,-entry[6])
    else:
        if entry[6]>-maxheap[0]: heapq.heappush(minheap,entry[6])
        else: heapq.heappush(minheap, -heapq.heappushpop(maxheap, -entry[6]))
    
    #trading_volume.append(entry[6])
    #break
    
median = (minheap[0]-maxheap[0]) // 2 if len(minheap) == len(maxheap) else minheap[0]

print('####RESULTS####')
print('For the year 2017:\n  Highest opening price: ',highest_open_price,'\n  Lowest opening price: ',lowest_open_price,
      '\n  Largest change in any one day: ',largest_change,
      '\n  Largest change between any two working days based on closing price: ',largest_change2,
     '\n  Average daily trading volume: ',total_volume/total_days,
     '\n  Median trading volume: ',median)

####RESULTS####
For the year 2017:
  Highest opening price:  53.11 
  Lowest opening price:  34.0 
  Largest change in any one day:  2.8100000000000023 
  Largest change between any two working days based on closing price:  35.8 
  Average daily trading volume:  89124.33725490196 
  Median trading volume:  76286.0


In [105]:
median, sorted(trading_volume)[middle], middle, len(trading_volume)

(76286.0, 76286.0, 127, 255)

In [99]:
def median(x):
    n = len(x)
    minheap = []
    maxheap = []
    middle = n//2
    
    #maxheap, element, minheap
    for num in x:
        if len(maxheap)<middle:
            heapq.heappush(maxheap,-num)
        else:
            if num>-maxheap[0]: heapq.heappush(minheap,num)
            else: heapq.heappush(minheap, -heapq.heappushpop(maxheap, -num))
    return (minheap[0]-maxheap[0]) // 2 if len(minheap) == len(maxheap) else minheap[0]
 
median([1,5,9,11,52])

9